## 데이터 스케일링

https://datascienceschool.net/view-notebook/f43be7d6515b48c0beb909826993c856/

### 생성되는 결과파일
datas 폴더에 생성됨
1. row_data_train.csv : 스케일 조정된 train 결과 (y_label 포함)
2. row_data_test.csv : 스케일 조정된 test 결과 (y_label 포함)

In [2]:
import pandas as pd
import numpy as np

## 전처리된 데이터 가져오기

In [65]:
data = pd.read_csv('./datas/raw_all_preprocessing.csv', header=0)  
cols = data.columns.values
cols = cols[1:].tolist()  # 주의! 


# 해당연도 추출
data['year'] = data.key.map(lambda x : str(x)[0:4]) 

# 문자열 합치기
data['split_key'] = data[['year', 'industry']].apply(lambda x: ''.join(x.astype(str)), axis=1)


# # 스케일 조정할 factor들만 셋팅
# data = data.reindex(cols,axis = 1)
data.head()


,Unnamed: 0,key,industry,label,S430000600O0,3001104004O0,3001212005O0,3001403004O0,3001111005O0,S430000100O0,...,DR00000159,DR00000160,DR00000161,DR00000162,DR00000163,DR00000164,DR00000165,DR00000166,year,split_key
0,0,2013000020,C,0,11.716837,2.969817,0.802569,0.416889,-1.886927,2.225912,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,2013C
1,1,2013000040,C,0,10.864216,2.969817,2.881315,2.845274,-1.886927,2.225912,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,2013C
2,2,2013000050,C,0,12.607617,2.969817,2.881315,2.845274,-0.363248,-0.851603,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,2013C
3,3,2013000070,M,0,13.337241,2.969817,2.881315,2.845274,-1.886927,-0.851603,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2013,2013M
4,4,2013000080,C,0,14.263528,2.969817,2.881315,0.095426,-0.456624,-0.851603,...,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2013,2013C


In [66]:
len(data)

7626

In [9]:
data.describe

<bound method NDFrame.describe of       S430000600O0  3001104004O0  3001212005O0  3001403004O0  3001111005O0  \
0        11.716837      2.969817      0.802569      0.416889     -1.886927   
1        10.864216      2.969817      2.881315      2.845274     -1.886927   
2        12.607617      2.969817      2.881315      2.845274     -0.363248   
3        13.337241      2.969817      2.881315      2.845274     -1.886927   
4        14.263528      2.969817      2.881315      0.095426     -0.456624   
5        14.560919      2.969817      2.881315      2.845274     -1.886927   
6        14.640227      2.969817      0.010336      2.845274     -0.368731   
7        12.517667      2.969817      2.881315      2.845274     -0.456624   
8        14.983315      2.969817      2.881315      2.845274     -0.245819   
9        11.473727      2.969817      2.881315      2.845274     -1.886927   
10       15.036512      2.969817      2.881315      2.845274     -0.013701   
11       10.730947      2.9698

## 데이터셋 분리하기

In [72]:
# 데이터셋 분리

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# 연도별 산업별 (split_key) 기준 분리
temp = data[data.label == 0]
distnct_key = temp['split_key'].drop_duplicates()



# 1) 정상
result_test = pd.DataFrame()
result_train = pd.DataFrame()

# 연도별 산업별 데이터 루프
for x in distnct_key:
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    temp = data.query("(split_key == '" + x + "') and (label == 0)")
    X = temp.iloc[:,:].values
    X_train, X_test, = train_test_split(X, 
                                            test_size=0.3, # train에 복제 데이터 추가될 예정이므로 0.3
                                            random_state=42) 

    train_df = pd.DataFrame(X_train)
    test_df = pd.DataFrame(X_test)
    result_train = [result_train, train_df]
    result_train = pd.concat(result_train) 
    result_test = [result_test, test_df]
    result_test = pd.concat(result_test)



# 2) 부실
result_test_y = pd.DataFrame()
result_train_y = pd.DataFrame()

# 산업별 분리는 무리, 연도별 적용
temp = data[data.label == 1]
distnct_key = temp['year'].drop_duplicates()


for x in distnct_key:
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    temp = data.query("(year == '" + x + "') and (label == 1)")


    Y = temp.iloc[:,:].values
    Y_train, Y_test, = train_test_split(Y, 
                                            test_size=0.3, # train에 복제 데이터 추가될 예정이므로 0.3
                                            random_state=42) 


    train_df = pd.DataFrame(Y_train)
    test_df = pd.DataFrame(Y_test)
    result_train_y = [result_train_y, train_df]
    result_train_y = pd.concat(result_train_y) 
    result_test_y = [result_test_y, test_df]
    result_test_y = pd.concat(result_test_y)

    
    

In [73]:
result_train_y.head()

,0,1,2,3,4,5,6,7,8,9,...,698,699,700,701,702,703,704,705,706,707
0,691,2013036210,C,1,10.9441,2.96982,2.88132,2.84527,-0.456624,2.22591,...,3,0,3,3,0,0,0,0,2013,2013C
1,281,2013007390,C,1,10.0089,2.96982,2.88132,2.84527,-0.456624,2.22591,...,3,3,0,3,0,3,0,2,2013,2013C
2,722,2013037950,C,1,12.5665,2.96982,2.88132,2.84527,-1.88693,2.22591,...,2,3,0,3,2,2,0,2,2013,2013C
3,754,2013039850,G,1,9.54853,2.96982,2.88132,2.84527,-1.88693,2.22591,...,3,1,0,3,0,3,0,0,2013,2013G
4,775,2013041960,C,1,12.8091,2.96982,2.88132,2.84527,-0.456624,2.22591,...,3,1,0,0,0,0,0,0,2013,2013C


In [74]:
len(data), len(result_train)+len(result_test), len(data[data.label == 0])

(7626, 7191, 7191)

In [75]:
len(data), len(result_train_y)+len(result_test_y), len(data[data.label == 1])

(7626, 435, 435)

## 데이터 셔플하기

In [19]:
# 데이터 셔플
from sklearn.utils import shuffle

for x in range(0,200):
    data = shuffle(data, random_state=42)

data.head()

,Unnamed: 0,key,industry,label,S430000600O0,3001104004O0,3001212005O0,3001403004O0,3001111005O0,S430000100O0,...,DR00000158,DR00000159,DR00000160,DR00000161,DR00000162,DR00000163,DR00000164,DR00000165,DR00000166,split_key
4889,4889,2016022100,M,0,13.686268,2.969817,0.001595,2.845274,-1.886927,2.225912,...,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2016M
1491,1491,2014001820,C,0,10.562794,2.969817,2.881315,2.845274,-0.456624,2.225912,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014C
563,563,2013025820,C,0,10.644710,2.969817,2.881315,2.845274,-0.456624,2.225912,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013C
4195,4195,2015102120,C,0,11.564398,2.969817,2.881315,2.845274,-1.886927,2.225912,...,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2015C
5134,5134,2016041440,C,0,12.096029,2.969817,2.881315,2.845274,-1.886927,2.225912,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016C


In [10]:
s_train_df.rename(columns = lambda x : cols[int(x)], inplace = True) # 항목명변경
s_test_df.rename(columns = lambda x : cols[int(x)], inplace = True) # 항목명변경
s_train_df.head()

,code,y_label,var15_3,var16_3,var16_2,var16_1,var18_3,var18_2,var18_1,var21_3,...,report_year2_04,report_year2_07,report_year2_10,report_year2_13,report_year3_01,report_year3_02,report_year3_04,report_year3_07,report_year3_10,report_year3_13
0,004310,0,12,13,14,13,6,6,9,17,...,1,0,0,0,0,0,1,0,0,0
1,033290,0,20,1,19,18,3,18,19,8,...,0,0,0,0,0,0,0,0,0,0
2,067290,0,4,2,3,2,5,6,7,2,...,0,0,0,0,0,0,0,0,0,0
3,090470,0,20,20,20,20,20,20,20,4,...,0,0,0,1,0,0,0,0,1,1
4,182690,0,3,3,2,18,2,2,2,2,...,0,0,0,2,0,0,0,0,0,3


In [11]:
# 파일생성
s_train_df.to_csv('./datas/suffled_train.csv', encoding='utf-8')
s_test_df.to_csv('./datas/suffled_test.csv', encoding='utf-8')

In [12]:
# 이미지 생성에 사용하지 않는 항목제거
X_train = np.array(s_train_df.iloc[:,2:].values) 
X_test = np.array(s_test_df.iloc[:,2:].values )

In [13]:
len(X_train[0])  # 항목수 확인

209

## 데이터 스케일 조정

https://datascienceschool.net/view-notebook/f43be7d6515b48c0beb909826993c856/

In [14]:
# 재작업시 위에 실행 후 처리하기

In [15]:
# 데이터 스케일 조정 (표준)

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)


In [16]:
# 데이터 스케일 조정 (MinMax)
# 재작업시 위에 실행 후 처리하기

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 255))
scaler.fit_transform(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [17]:
# 확인

In [18]:
print(len(X_train),X_train[0].min(),X_train[0].max())

1388 0.0 241.57894736842104


In [19]:
print(len(X_train),X_train[-1].min(),X_train[-1].max())

1388 0.0 255.0


In [20]:
print(len(X_train),X_train.min(),X_train.max())

1388 0.0 255.0


In [21]:
print(len(X_test),X_test.min(),X_test.max())

596 0.0 255.0


## x, y재병합
https://datascienceschool.net/view-notebook/7002e92653434bc88c8c026c3449d27b/

y 별도로 쪼갠 후 X만 스케일링하고 다시 합치는 단계

In [22]:
cols = data.columns.values.tolist()
len(cols)

211

In [23]:
cols = data.columns.values.tolist() # features 항목명 주의

# train data, y_label 재병합
temp_x1 = pd.DataFrame()
temp_x1[0] = s_train_df['code']
temp_x1[1] = y_train.tolist()
temp_x2 = pd.DataFrame(X_train)
temp_y = pd.DataFrame()
temp_y['y_label'] = y_train.tolist()
train_df = pd.concat([temp_x1,temp_x2], axis=1)
train_df.rename(columns = lambda x : cols[int(x)], inplace = True) # 항목명변경
print(len(temp_x1), len(temp_x2), "->", len(train_df))

# test data, y_label 재병합
temp_x1 = pd.DataFrame()
temp_x1[0] = s_test_df['code']
temp_x1[1] = y_test.tolist()
temp_x2 = pd.DataFrame(X_test)
temp_y = pd.DataFrame()
temp_y['y_label'] = y_test.tolist()
test_df = pd.concat([temp_x1,temp_x2], axis=1)
test_df.rename(columns = lambda x : cols[int(x)], inplace = True) # 항목명변경
print(len(temp_x1), len(temp_x2), "->", len(test_df))

1388 1388 -> 1388
596 596 -> 596


In [24]:
test_df.head()

,code,y_label,code,y_label,var15_3,var16_3,var16_2,var16_1,var18_3,var18_2,...,report_year2_01,report_year2_02,report_year2_04,report_year2_07,report_year2_10,report_year2_13,report_year3_01,report_year3_02,report_year3_04,report_year3_07
0,067280,0,161.052632,147.631579,174.473684,147.631579,26.842105,214.736842,187.894737,53.684211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.5,0.0
1,041440,0,161.052632,161.052632,187.894737,174.473684,174.473684,187.894737,187.894737,187.894737,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,017900,0,228.157895,214.736842,174.473684,120.789474,67.105263,80.526316,93.947368,174.473684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,001570,0,214.736842,255.000000,174.473684,13.421053,134.210526,53.684211,241.578947,40.263158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,008870,0,174.473684,147.631579,120.789474,107.368421,134.210526,134.210526,120.789474,228.157895,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
print ("train 부실비중 : ", train_df["y_label"].sum()/len(train_df["y_label"]) * 100,", test 부실비중 : ",  test_df["y_label"].sum()/len(test_df["y_label"]) * 100)

train 부실비중 :  y_label        3.242075
y_label    12754.834673
dtype: float64 , test 부실비중 :  y_label        2.516779
y_label    12824.311197
dtype: float64


## 파일 생성

In [26]:
# 파일생성
train_df.to_csv('./datas/row_data_train.csv', encoding='utf-8')
test_df.to_csv('./datas/row_data_test.csv', encoding='utf-8')
train_df.head()

,code,y_label,code,y_label,var15_3,var16_3,var16_2,var16_1,var18_3,var18_2,...,report_year2_01,report_year2_02,report_year2_04,report_year2_07,report_year2_10,report_year2_13,report_year3_01,report_year3_02,report_year3_04,report_year3_07
0,004310,0,147.631579,161.052632,174.473684,161.052632,67.105263,67.105263,107.368421,214.736842,...,31.875,0.0,0.0,0.0,0.0,0.0,25.5,0.0,0.0,0.000
1,033290,0,255.000000,0.000000,241.578947,228.157895,26.842105,228.157895,241.578947,93.947368,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
2,067290,0,40.263158,13.421053,26.842105,13.421053,53.684211,67.105263,80.526316,13.421053,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
3,090470,0,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,40.263158,...,0.000,0.0,0.0,15.0,0.0,0.0,0.0,0.0,127.5,10.625
4,182690,0,26.842105,26.842105,13.421053,228.157895,13.421053,13.421053,13.421053,13.421053,...,0.000,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,31.875
